In [5]:
# import modules + data
from plotly.subplots import make_subplots
import pandas as pd
import plotly.graph_objects as go

In [6]:
# graphing functions to make below less difficult
def bubble(x, y, color, size, labels):
    data = [go.Scatter (
        x = x,
        y = y,
        text=labels,
        mode = 'markers',
        marker = dict(
            color=color,
            size=size,
            sizemode='area',
            sizeref=2.*max(size)/(45.**2),
            sizemin=4))]
    fig = go.Figure(data)
    fig.show()

def bar(x, y, colors):
    trace1 = go.Bar(
        x=x,
        y=y,
        marker_color=colors)
    fig = go.Figure([trace1])
    fig.show()
    
def barStack(labels, y1, y2):
    trace1 = go.Bar(
        x=labels,
        y=y1)
    trace2 = go.Bar(
        x=labels,
        y=y2)
    fig = go.Figure([trace1, trace2])
    fig.update_layout(barmode='stack')
    fig.show()    


In [7]:
data = pd.read_csv('SPopulation.csv')

@TODO: merge state abbrevs

# check how many parties there are and what they are. and if any new footnotes are up
data['Party'].unique()

array(['Republican', 'Democratic', 'Independent'], dtype=object)

In [8]:
# set up what party gets what colors and transform party column in data into a corresponding list of colors
colordict = {'Republican': 'red', 'Democratic':'blue', 'Independent':'gray', 'Vacant':'purple'}
colors = [colordict[k] for k in list(data['Party'])]
data['color'] = colors

In [9]:
sorts = data.sort_values(by='Population').reset_index(drop=True)
sorts.head()

,State,Senator,Party,Born,Assumed office,Term up,Population,color
0,Wyoming,Cynthia Lummis,Republican,(age 66),"January 3, 2021",2026,578759,red
1,Wyoming,John Barrasso,Republican,(age 68),"June 25, 2007[y]",2024,578759,red
2,Vermont,Bernie Sanders,Independent,(age 79),"January 3, 2007",2024,623989,gray
3,Vermont,Patrick Leahy,Democratic,(age 80),"January 3, 1975",2022,623989,blue
4,Alaska,Lisa Murkowski,Republican,(age 63),"December 20, 2002[d]",2022,731545,red


In [13]:
# very simple graph showing how many people democrats vs republicans represent
mylabels = ['Republican', 'Democrat']

all_Rs = sorts[sorts['Party'] == 'Republican']['Population'].sum() / 2
all_Ds = sorts[sorts['Party'] != 'Republican']['Population'].sum() / 2 # dividing by 2 because 2 senators per state

bar(mylabels, [all_Rs, all_Ds], ['red', 'blue']) 

# @TODO: Include independents on top of democrats in a different color

In [14]:
sorts.head()

,State,Senator,Party,Born,Assumed office,Term up,Population,color
0,Wyoming,Cynthia Lummis,Republican,(age 66),"January 3, 2021",2026,578759,red
1,Wyoming,John Barrasso,Republican,(age 68),"June 25, 2007[y]",2024,578759,red
2,Vermont,Bernie Sanders,Independent,(age 79),"January 3, 2007",2024,623989,gray
3,Vermont,Patrick Leahy,Democratic,(age 80),"January 3, 1975",2022,623989,blue
4,Alaska,Lisa Murkowski,Republican,(age 63),"December 20, 2002[d]",2022,731545,red


In [ ]:
def barStack(labels, y1, y2):
    trace1 = go.Bar(
        x=labels,
        y=y1)
    trace2 = go.Bar(
        x=labels,
        y=y2)
    fig = go.Figure([trace1, trace2])
    fig.update_layout(barmode='stack')
    fig.show()    

In [20]:
# Should this not be with the barstack function


# Stacked bar chart (for 2 senators per state)

# since data is sorted by population, senators from the same state must be adjacent
# so taking every other one gives one from each state
stack_1 = sorts[sorts.index % 2 == 0]
stack_2 = sorts[sorts.index % 2 == 1]

# colors stuff, as above
# colors1 = [colordict[k] for k in list(stack_1['Party'])]
# colors2 = [colordict[k] for k in list(stack_2['Party'])]


# bottom half of bar chart
trace1 = go.Bar(
    x=list(stack_1['State']),
    y=list(stack_1['Population']/2), # divide state pop in two so two senators makes the right pop come out
    # But don't want hovering to SHOW this
    marker_color = stack_1['color'],
    text=list(stack_1['Senator'])) # hovering shows state

# top half, same thing
trace2 = go.Bar(
    x=list(stack_1['State']), # maybe do abbreviations? 
    y=list(stack_2['Population']/2),
    marker_color = stack_2['color'],
    hovertext=list(stack_2['Senator'])) 

# make chart!
fig = go.Figure([trace1])
fig.add_trace(trace2)

# add a line between 9 and 10 to show the halfway point
# @TODO: make robust against different distributions of population
fig.add_vline(x=9.5, line_dash='dot')

fig.update_layout(barmode='stack', 
                  xaxis={'categoryorder':'total descending'}, 
                  showlegend = False,
                 title = "# of People each Senator Represents",
                 template = 'plotly_white')
fig.show()    

In [21]:
# Polar chart - Senate chamber style
# SETUP

# specifies two side-by-side subplots
fig = make_subplots(rows=1, cols=2, specs=[[{'type': 'polar'}]*2]*1, horizontal_spacing=0)

# put coords in table?
rep_coords = [(r, a*90 / (r+2)) for a in range(12) for r in range(max(5, a-2),10)]
dem_coords = [(r, 180 - a*90 / (r+2)) for a in range(12) for r in range(max(5, a-2),10)]


Ds = sorts[sorts['Party'] != 'Republican']
Rs = sorts[sorts['Party'] == 'Republican']

Ds.head()

,State,Senator,Party,Born,Assumed office,Term up,Population,color
2,Vermont,Bernie Sanders,Independent,(age 79),"January 3, 2007",2024,623989,gray
3,Vermont,Patrick Leahy,Democratic,(age 80),"January 3, 1975",2022,623989,blue
10,Delaware,Chris Coons,Democratic,(age 57),"November 15, 2010[i]",2026,973764,blue
11,Delaware,Tom Carper,Democratic,(age 74),"January 3, 2001",2024,973764,blue
12,Rhode Island,Jack Reed,Democratic,(age 71),"January 3, 1997",2026,1059361,blue


In [23]:


# # ....uhh is this.... used?
# r_vals = [a[0] for a in dem_coords] + [a[0] for a in rep_coords]
# theta_vals = [a[1] for a in dem_coords] + [a[1] for a in rep_coords]




# democrat side
fig.add_trace(go.Scatterpolar(
        r = [a[0] for a in dem_coords],
        theta = [a[1] for a in dem_coords],
        text=list(Ds['Senator']),
        mode = 'markers',
        marker = dict(
            color = Ds['color'],
            size = list(Ds['Population']),
            sizemode = 'area',
            sizeref = 2.*40000000/(50.**2),
            sizemin = 4)
    ), 1, 1)

# republican side
fig.add_trace(go.Scatterpolar(
        r = [a[0] for a in rep_coords],
        theta = [a[1] for a in rep_coords],
        text = list(Rs['Senator']),
        mode = 'markers',
        marker = dict(
            color = Rs['color'],
            size = list(Rs['Population']),
            sizemode = 'area',
            sizeref = 2.*40000000/(50.**2),
            sizemin = 4)
    ), 1, 2)

# @TODO: Draw line around 18 top senators


fig.update_layout(
    title = "US Senate",
    font_size = 15,
    showlegend = False,
    polar = dict(
        sector = [90,180],
        bgcolor = "white",
        angularaxis = dict(showline=False,showticklabels=False, ticks=''),
        radialaxis = dict(showline=False, showticklabels=False, ticks='')
    ),
    polar2 = dict(
        sector = [0, 90],
        bgcolor = "white",
        angularaxis = dict(showline=False,showticklabels=False, ticks=''),
        radialaxis = dict(showline = False,showticklabels=False, ticks='')
    ),
    paper_bgcolor = "white"
)



fig.show()

In [ ]:
# OLD
R_coords = []
for i in range(10):
    for j in range(5):
        R_coords.append([i, j])

D_coords = []
for i in range(10, 20):
    for j in range(5):
        D_coords.append([i, j])
        
# put coords into table?

In [ ]:
# bubble plot of population of each state, with mouseover labels for each senator
x_vals = [a[0] for a in (R_coords + D_coords)]
y_vals = [a[1] for a in (R_coords + D_coords)]
pops = [a for a in (R_pops + D_pops)]

bubble(x_vals, y_vals, colors, pops, full_sen_list)